## Incremental learning with River

In [58]:
from river import active
from river import datasets
import numpy as np
from river import feature_extraction
from river import linear_model
from river import metrics
import sys
sys.path.append('.\\..\\..\\..\\Chapter2-Z24-dataset')
from predict import binarize
import os
from sklearn.neighbors import KNeighborsClassifier

In [6]:
from river import active
from river import datasets
import numpy as np
from river import feature_extraction
from river import linear_model
from river import metrics
import sys
sys.path.append('.\\Chapter2-Z24-dataset')

dataset = datasets.SMSSpam()

metric = metrics.Accuracy()
model = (
    feature_extraction.TFIDF(on='body') |
    linear_model.LogisticRegression()
)
model = active.EntropySampler(model, seed=42)

n_samples_used = 0
for x, y in dataset:
    y_pred, ask = model.predict_one(x)
    metric = metric.update(y, y_pred)
    if ask:
        n_samples_used += 1
        model = model.learn_one(x, y)

metric

<class 'numpy.ndarray'>


Accuracy: 86.60%

In [10]:
x = np.array(dataset)
dataset["Samples"]

TypeError: 'SMSSpam' object is not subscriptable

In [3]:
for xi, yi in zip(X, y):
    # This is where the model learns
    pass


In [8]:
xi.shape

(30,)

In [7]:
X[0].shape

(30,)

In [9]:
from river import stream

for xi, yi in stream.iter_sklearn_dataset(datasets.load_breast_cancer()):
    pass


In [13]:
xi

{'mean radius': 7.76,
 'mean texture': 24.54,
 'mean perimeter': 47.92,
 'mean area': 181.0,
 'mean smoothness': 0.05263,
 'mean compactness': 0.04362,
 'mean concavity': 0.0,
 'mean concave points': 0.0,
 'mean symmetry': 0.1587,
 'mean fractal dimension': 0.05884,
 'radius error': 0.3857,
 'texture error': 1.428,
 'perimeter error': 2.548,
 'area error': 19.15,
 'smoothness error': 0.007189,
 'compactness error': 0.00466,
 'concavity error': 0.0,
 'concave points error': 0.0,
 'symmetry error': 0.02676,
 'fractal dimension error': 0.002783,
 'worst radius': 9.456,
 'worst texture': 30.37,
 'worst perimeter': 59.16,
 'worst area': 268.6,
 'worst smoothness': 0.08996,
 'worst compactness': 0.06444,
 'worst concavity': 0.0,
 'worst concave points': 0.0,
 'worst symmetry': 0.2871,
 'worst fractal dimension': 0.07039}

In [12]:
from river import active
from river import datasets
import numpy as np
from river import feature_extraction
from river import linear_model
from river import metrics

# Load the dataset
dataset = datasets.SMSSpam()

# Convert the dataset to numpy arrays
data = [(x, y) for x, y in dataset]
X = np.array([item[0]['body'] for item in data])
y = np.array([item[1] for item in data])

metric = metrics.Accuracy()
model = (
    feature_extraction.TFIDF() |
    linear_model.LogisticRegression()
)
model = active.EntropySampler(model, seed=42)

n_samples_used = 0
for i in range(X.shape[0]):
    x = X[i]
    y_true = y[i]
    y_pred, ask = model.predict_one(x)
    metric = metric.update(y_true, y_pred)
    if ask:
        n_samples_used += 1
        model = model.learn_one(x, y_true)

print(metric)


Accuracy: 86.60%


In [23]:
from sklearn import datasets
iris = datasets.load_iris()
from river import active
from river import datasets
import numpy as np
from river import feature_extraction
from river import linear_model
from river import metrics



# Load the dataset
dataset = datasets.SMSSpam()

# Convert the dataset to numpy arrays
data = [(x, y) for x, y in dataset]
X = iris['data']#np.array([item[0]['body'] for item in data])
y = iris['target'] #np.array([item[1] for item in data])

metric = metrics.Accuracy()
model = (
    
    linear_model.LogisticRegression()
)
model = active.EntropySampler(model, seed=42)

n_samples_used = 0
for i in range(X.shape[0]):
    x = X[i]
    y_true = y[i]
    y_pred, ask = model.predict_one(x)
    metric = metric.update(y_true, y_pred)
    if ask:
        n_samples_used += 1
        model = model.learn_one(x, y_true)

print(metric)


ValueError: Unsupported type for data: <class 'numpy.ndarray'>

In [20]:
type(iris['target'][0])

numpy.int32

In [29]:
iris['data'].shape

(150, 4)

In [26]:
from sklearn import datasets
from river import active
from river import datasets as river_datasets
from river import linear_model
from river import metrics

# Load the iris dataset from sklearn
sklearn_iris = datasets.load_iris()
X_np = sklearn_iris['data']
y_np = sklearn_iris['target']

# Convert the numpy arrays to a list of dictionaries
X = [{"feature_" + str(j): X_np[i][j] for j in range(X_np.shape[1])} for i in range(X_np.shape[0])]

# Initialize model and metric
metric = metrics.Accuracy()
model = linear_model.LogisticRegression()
model = active.EntropySampler(model, seed=42)

n_samples_used = 0
for x, y_true in zip(X, y_np):
    y_pred, ask = model.predict_one(x)
    metric = metric.update(y_true, y_pred)
    if ask:
        n_samples_used += 1
        model = model.learn_one(x, y_true)

print(metric)
print(n_samples_used)

Accuracy: 44.00%
41


In [62]:
from sklearn import datasets as skdatasets  # Import to avoid naming conflicts
from river import active
from river import datasets
from river import linear_model
from river import metrics

# Load the iris dataset from sklearn
iris_sk = skdatasets.load_iris()

# Prepare the data in a format compatible with river
X_river = [
    {f"feature_{i}": value for i, value in enumerate(sample)}
    for sample in iris_sk['data']
]
y_river = iris_sk['target'].tolist()  # Converting to list for easier iteration

metric = metrics.Accuracy()
base_model = linear_model.LogisticRegression()
model = active.EntropySampler(base_model, seed=42)

n_samples_used = 0

# Process each sample in the dataset
for i in range(len(X_river)):
    x = X_river[i]
    y_true = y_river[i]
    y_pred, ask = model.predict_one(x)  # x is now a dictionary
    metric = metric.update(y_true, y_pred)

    if ask:
        n_samples_used += 1
        model = model.learn_one(x, y_true)  # Learn from the true label

print(metric)
print(f"Number of samples used for learning: {n_samples_used}")

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
Accuracy: 44.00%
Number of samples used for learning: 41


In [37]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# 1. Load the iris dataset
iris = datasets.load_iris()
X = iris.data
y = iris.target

# 2. Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Train a logistic regression model
clf = LogisticRegression()  # Setting a high max_iter to ensure convergence
clf.fit(X_train, y_train)

# 4. Evaluate the model's performance on the test set
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")


Accuracy: 1.0000


c:\Users\amroa\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [45]:
X_train = np.load("C:\\Users\\amroa\\Documents\\thesis\\X_train.npy")
labels_train = np.load("C:\\Users\\amroa\\Documents\\thesis\\labels_train.npy")
X_test = np.load("C:\\Users\\amroa\\Documents\\thesis\\X_test.npy")
labels_test = np.load("C:\\Users\\amroa\\Documents\\thesis\\labels_test.npy")

In [47]:
X_train.shape, \
X_test.shape

((544, 906), (68, 906))

In [60]:
# define hyperparameters
features_to_keep = 79
neighbors = 41

y_train = binarize(labels_train)
log_reg = LogisticRegression(C = 12.5) 
y_train = binarize(labels_train)
log_reg.fit(X_train, y)
sorted_feature_indices = np.argsort(log_reg.coef_[0])[::-1] # get the indices of the most important features in descending order

In [59]:
# dimensionality reduction (to stabilize KNN and avoid dimensionality curse)
indices = sorted_feature_indices[:features_to_keep]
X_train_top_feats = X_train[:, indices]

In [63]:
from sklearn import datasets as skdatasets  # Import to avoid naming conflicts
from river import active
from river import datasets
from river import linear_model
from river import metrics

# Prepare the data in a format compatible with river
X_river = [
    {f"feature_{i}": value for i, value in enumerate(sample)}
    for sample in X_train_top_feats
]
y_river = y_train.tolist()  # Converting to list for easier iteration

metric = metrics.Accuracy()
base_model = linear_model.LogisticRegression()
model = active.EntropySampler(base_model, seed=42)

n_samples_used = 0

# Process each sample in the dataset
for i in range(len(X_river)):
    x = X_river[i]
    y_true = y_river[i]
    y_pred, ask = model.predict_one(x)  # x is now a dictionary
    metric = metric.update(y_true, y_pred)

    if ask:
        n_samples_used += 1
        model = model.learn_one(x, y_true)  # Learn from the true label

print(metric)
print(f"Number of samples used for learning: {n_samples_used}")

Accuracy: 97.06%
Number of samples used for learning: 188


### Testing phase

In [ ]:
X_river_test = [
    {f"feature_{i}": value for i, value in enumerate(sample)}
    for sample in X_train_top_feats
]
y_river = y_train.tolist()  # Converting to list for easier iteration


# Process each sample in the dataset
for i in range(len(X_river)):
    x = X_river[i]
    y_true = y_river[i]
    y_pred, ask = model.predict_one(x)  # x is now a dictionary
    metric = metric.update(y_true, y_pred)

    if ask:
        n_samples_used += 1
        model = model.learn_one(x, y_true)  # Learn from the true label

print(metric)
print(f"Number of samples used for learning: {n_samples_used}")